In [ ]:
import bz2
import json
from pathlib import Path

import pandas as pd

my_dir = Path('.')


def _stream_lines(path: Path = my_dir / 'lyrics-distinct-emi-2021-10-15.json.bz2'):
    with bz2.open(path, 'r') as zipped_file:
        for line in zipped_file:
            yield json.loads(line)

In [ ]:
dic = {'ID': [], 'Genre': [], 'Popularity': [], 'Date': []}
counter = 0

for elem in _stream_lines():
    # print(elem)
    # break
    skip_loop = False

    # get date
    for i in elem['release']:
        if type(elem['release'][i]) is dict:
            skip_loop = True
            counter += 1
            break

        else:
            year, month, n = elem['release'][i].split('-')
            dic['Date'].append(int(year))

    if not skip_loop:
        # get ID
        name = str()
        for i in elem['_id']:
            n = elem['_id'][i]
            n = n.replace(' ', '')
            if len(name) == 0:
                name = n
            else:
                name = name + '_' + n
        dic['ID'].append(name)

        # get popularity
        dic['Popularity'].append(int(elem['popularity']))

        # get genre
        g = elem['genre']
        g = g.replace(' ', '_')
        dic['Genre'].append(g)

        # get features
        for f in elem['features']:
            if type(elem['features'][f]) is dict:
                for e in elem['features'][f]:
                    new_name = f + '_' + e
                    if new_name not in dic:
                        dic[new_name] = []
                        dic[new_name].append(elem['features'][f][e])
                    else:
                        dic[new_name].append(elem['features'][f][e])
            else:
                if f not in dic:
                    dic[f] = []
                    dic[f].append(elem['features'][f])
                else:
                    dic[f].append(elem['features'][f])

        # get affective scores
        for f in elem['affective']:
            if type(elem['affective'][f]) is dict:
                for e in elem['affective'][f]:
                    new_name = f + '_' + e
                    if new_name not in dic:
                        dic[new_name] = []
                        dic[new_name].append(elem['affective'][f][e])
                    else:
                        dic[new_name].append(elem['affective'][f][e])
            else:
                if f not in dic:
                    dic[f] = []
                    dic[f].append(elem['affective'][f])
                else:
                    dic[f].append(elem['affective'][f])
print(counter, 'track have been droped due to invalid date')


In [ ]:
# explore missing data
for k, v in dic.items():
    print(k)
    print(len(v))

In [ ]:
features_df = pd.DataFrame.from_dict(dic)

In [ ]:
features_df.to_csv(my_dir / 'features_distinct.csv', index=False)